In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import networkx
import Graph

g = Graph.buildGraph()
# mulGraph = Graph.buildGraph()

Connection pairs:
[
    (7529, 3491, 489.50118772441493),
    (7483, 6868, 128.34041442898507),
    (7487, 7090, 100.0674332727893),
    (7492, 6907, 44.872694234504756),
    (7497, 6861, 45.77161729321111),
    (7498, 6863, 15.894429251450745),
    (7504, 672, 71.5994991372535),
    (7511, 234, 11.285070412497834),
    (7516, 4166, 19.355601312460404),
    (7685, 7508, 26.251222182548965),
    (7487, 7485, 34.69950831729087),
    (7498, 7483, 133.01878867990203),
    (7499, 7489, 161.861689287105),
    (7683, 7518, 839.9571389347612)
]

7529 3491 time = 305.9382423277593 dis = 489.50118772441493 routeVar = Walk

7483 6868 time = 80.21275901811566 dis = 128.34041442898507 routeVar = Walk

7487 7090 time = 62.54214579549331 dis = 100.0674332727893 routeVar = Walk

7492 6907 time = 28.04543389656547 dis = 44.872694234504756 routeVar = Walk

...

7487 7485 time = 21.68719269830679 dis = 34.69950831729087 routeVar = Walk

7498 7483 time = 83.13674292493876 dis = 133.01878867990203 routeVar = Walk

7499 7489 time = 101.16355580444062 dis = 161.861689287105 routeVar = Walk

7683 7518 time = 524.9732118342257 dis = 839.9571389347612 routeVar = Walk

Number of subgraphs: 1

In [65]:
ug = g.to_undirected()
print(networkx.shortest_path(ug, 35, 7510))
# print(networkx.dijkstra_path(g, 35, 234))
print(networkx.dijkstra_path(g, 7511, 234))

[35, 377, 2568, 2696, 235, 233, 234, 7511, 7515, 7510]
[7511, 234]


In [ ]:
# Return connection pairs
from math import sqrt
from rtree import index
from rich import print

def connectGraph(subGraphs: list[networkx.MultiDiGraph]):
    connectionPair = []
    for i in range(len(subGraphs)):
        for j in range(i + 1, len(subGraphs)):
            idx = index.Index()
            
            nodeCoordI = networkx.get_node_attributes(subGraphs[i], 'coord')
            
            iNodes = list(subGraphs[i].nodes)
            jNodes = list(subGraphs[j].nodes)
            # print(len(list(subGraphs[i].nodes)))
            # print(subGraphs[i].nodes[7278]['coord'])
            # print(subGraphs[i].has_node(7276))
            
            for node in iNodes:
                idx.insert(node, (nodeCoordI[node][0], nodeCoordI[node][1], nodeCoordI[node][0], nodeCoordI[node][1]))
                
            # print(jNodes)
            minDisSoFar = -1
            connection1 = -1
            connection2 = -1
            for node in jNodes:
                # print(subGraphs[j].nodes[7529]['coord'])
                x, y = subGraphs[j].nodes[node]['coord']
                nearestId = list(idx.nearest((x, y, x, y), 1))[0]
                x1, y1 = nodeCoordI[nearestId]
                
                dis = sqrt((x - x1)**2 + (y - y1)**2)
                if minDisSoFar == -1 or minDisSoFar > dis:
                    minDisSoFar = dis
                    connection1 = node
                    connection2 = nearestId
                    
            
            if connection1 == 1:
                print("No points found")
                return
            
            # Only connect two stops that are less than 1km apart
            if minDisSoFar < 1000:
                connectionPair.append((connection1, connection2))
                
    return connectionPair
    

connectGraph(mulGraph)

In [ ]:
# Create edges
def createPath(graph: networkx.MultiDiGraph, stopIdNode: list[str], stopPoints: list[tuple], pathPoints: list[tuple], runningTime, routeVarId):
    pathIndex = -1
    totalLength = LineString(pathPoints).length
    for i in range(len(stopIdNode)):
        if pathIndex == -1:
            pathIndex += 1
            continue
        
        closestIndex = findNextClosest(pathIndex, stopPoints[i], pathPoints)
        line = LineString(pathPoints[pathIndex : closestIndex + 1])
        pathIndex = closestIndex
        # print(stopIdNode[i - 1], stopIdNode[i], line.length)
        graph.add_edge(stopIdNode[i - 1], stopIdNode[i], time = runningTime / totalLength * line.length, dis = line.length, routeVar = routeVarId)
        networkx.set_node_attributes(graph, {i - 1: {"coord": stopPoints[i - 1]}})
        networkx.set_node_attributes(graph, {i: {"coord": stopPoints[i]}})
        
    return graph

In [ ]:
# Get nodes attributes
from rich import print
timeLabel = networkx.get_edge_attributes(g, 'time')
print(g.nodes[35]['coord'])
print(g[7526][7529][0]['time'])

In [5]:
# List of subgraphs
from networkx import connected_components


ug = g.to_undirected()
# k = g.subgraph(list(connected_components(ug))[3])

subGraphs = [g.subgraph(com) for com in list(component for component in list(connected_components(ug)))]
print(len(subGraphs))
# print(subGraphs[0])
# [g.subgraph(component for component in list(connected_components(ug)) )]
# gs = [sub for sub in ]
# print(k)
# print(k.nodes)

# networkx.draw_spring(k, with_labels = True)



1


In [ ]:
# Correct Dijkstra
from rich import print
def Dijkstra1(g: networkx.MultiDiGraph, index = 0):
    minHeap = [[0, list(g.nodes)[index], list(g.nodes)[index]]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    for i in list(g.nodes):
        if i not in shortestPath:
            shortestPath[i] = (-1, -1) 
            
    return shortestPath

shortest = Dijkstra1(g, 0)
print(shortest) 

In [2]:
import heapq

def DijkstraFromTo(g: networkx.MultiDiGraph, start, end):
    minHeap = [[0, start, start]]
    shortestPath = {}
    while len(minHeap) > 0:
        weight, cur, fro = heapq.heappop(minHeap)
        
        if cur in shortestPath:
            continue
        
        shortestPath[cur] = (fro, weight)
        
        if cur == end:
            break
        
        for u in networkx.neighbors(g, cur):
            if u not in shortestPath:
                heapq.heappush(minHeap, [weight + g[cur][u][0]['time'], u, cur])
    
    if end not in shortestPath:
        return -1
    
    return shortestPath[end][1]
    
DijkstraFromTo(g, 7526, 7529)

7.978503198318191

In [27]:
print(networkx.is_path(g, [7278, 31]))

False

In [7]:
from rich.progress import track
from rich import print
import time
import json

def nodeNet(g: networkx.MultiDiGraph, node):
    shortcuts = 0
    shortcutPairs = []
    pre = g.pred[node]
    succ = g.succ[node]
    for u in pre:
        for w in succ:
            if networkx.is_path(g, [u, w]):
                continue
            if pre[u][0]['time'] + succ[w][0]['time'] == DijkstraFromTo(g, u, w):
                shortcuts += 1
                shortcutPairs.append((u, w))
                
    # print(f"{shortcuts = }, {len(pre) = }, {len(succ) = }")
    return shortcuts - (len(pre) + len(succ)), shortcutPairs

def contract(g: networkx.MultiDiGraph, x):
    stuff, edges = nodeNet(g, x)
    # print(edges)
    for edge in edges:
        # print(g.get_edge_data(edge[0], x))
        uPath = g.get_edge_data(edge[0], x)[0].get('path', [])
        uData = g.get_edge_data(edge[0], x)[0]
        wPath = g.get_edge_data(x, edge[1])[0].get('path', [])
        wData = g.get_edge_data(x, edge[1])[0]
        # print(f"{uPath = }, {wPath = }")
        # print(f"{uData = }, {wData = }")
        g.add_edge(edge[0], edge[1], time = uData['time'] + wData['time'], dis = uData['dis'] + wData['dis'], path = uData.get('path', []) + [x] + wData.get('path', []))
    

def contractionStage(d: networkx.MultiDiGraph):
    g: networkx.MultiDiGraph = d.copy()
    edgeDifference = {}
    nodeList = list(g.nodes)
    for x in nodeList:
        edgeDifference[x] = nodeNet(g, x)
        
    for x in track(reversed(nodeList)):
        start = time.time()
        x = min(edgeDifference, key=edgeDifference.get)
        for pre in g.pred[x]:
            if edgeDifference.get(pre, None) is None: 
                continue
            newTuple = (edgeDifference[pre][0] + 1, edgeDifference[pre][1])
            # print(newTuple)
            edgeDifference.pop(pre)
            edgeDifference[pre] = newTuple
        for succ in g.succ[x]:
            if edgeDifference.get(succ, None) is None: 
                continue
            newTuple = (edgeDifference[succ][0] + 1, edgeDifference[succ][1])
            edgeDifference.pop(succ)
            edgeDifference[succ] = newTuple
        contract(g, x)
        edgeDifference.pop(x)
        # print("Finish Node", x, "in", time.time() - start)
        # print("remove", x)
    
    print(g)
    return edgeDifference
    
ed = contractionStage(g)

Finish Node 7265 in 0.0010006427764892578

Finish Node 522 in 0.0009999275207519531

Finish Node 1384 in 0.0005104541778564453

Finish Node 30 in 0.0

Finish Node 7486 in 0.0010001659393310547

...

Finish Node 2444 in 0.3181626796722412

Finish Node 2553 in 0.4924914836883545

Finish Node 2594 in 0.3980560302734375

Finish Node 4062 in 0.7734413146972656

KeyboardInterrupt: 

In [17]:
print(min(ed, key=ed.get))
ed[min(ed, key=ed.get)][0] += 1

7265

TypeError: 'tuple' object does not support item assignment

In [2]:
di = networkx.floyd_warshall(g, 'time')

In [11]:
from rich.progress import track
import time
import time

calTime = {}

def FloydWarshall(g: networkx.MultiDiGraph):
    start = time.time()
    nodeList = list(g.nodes)
    
    adjArray = {}
    adjArray = {u: {v: (-1, -1) for v in nodeList} for u in nodeList}
    print("Empty adjacent dict in:", time.time() - start)
    start = time.time()
    
    for node in nodeList:
        adjArray[node][node] = (0, node)
        
    edgeList = list(g.edges)
    for u, v, trash in edgeList:
         adjArray[u][v] = (g[u][v][0]['time'], u)
    print("Only intermediate pairs in:", time.time() - start)
    print ("passed")
         
        
    # return
    nodeL = -1
    longestSoFar = -1
    for k in reversed(nodeList):
        print("At", k, end="")
        start = time.time()
        for i in nodeList:
            for j in nodeList:
                if i == j:
                    continue
                if k == i or k == j:
                    continue
                
                if adjArray[i][k][0] == -1 or adjArray[k][j][0] == -1:
                    continue
                
                if adjArray[i][j][0] == -1 or adjArray[i][j][0] > adjArray[i][k][0] + adjArray[k][j][0]:
                    adjArray[i][j] = (adjArray[i][k][0] + adjArray[k][j][0], k)
        
        dur = time.time() - start
        calTime[k] = dur
        if longestSoFar < dur:
            longestSoFar = dur
            nodeL = k
        print(" took:", dur, ", longest so far:", nodeL, "in:", longestSoFar)      
    return adjArray

adj = FloydWarshall(g)


Empty adjacent dict in: 1.634474277496338
Only intermediate pairs in: 0.4219071865081787
passed
At 7686 took: 2.0991671085357666 , longest so far: 7686 in: 2.0991671085357666
At 7685 took: 2.162587881088257 , longest so far: 7685 in: 2.162587881088257
At 7684 took: 2.0559885501861572 , longest so far: 7685 in: 2.162587881088257
At 7683 took: 1.4940078258514404 , longest so far: 7685 in: 2.162587881088257
At 7682 took: 1.5058503150939941 , longest so far: 7685 in: 2.162587881088257
At 7517 took: 1.5824236869812012 , longest so far: 7685 in: 2.162587881088257
At 7518 took: 1.5780808925628662 , longest so far: 7685 in: 2.162587881088257
At 7512 took: 1.4809050559997559 , longest so far: 7685 in: 2.162587881088257
At 7516 took: 1.5352087020874023 , longest so far: 7685 in: 2.162587881088257
At 7511 took: 1.4673316478729248 , longest so far: 7685 in: 2.162587881088257
At 7515 took: 1.5284993648529053 , longest so far: 7685 in: 2.162587881088257
At 7510 took: 1.4754865169525146 , longest so 

In [1]:
calTime['2252']

NameError: name 'calTime' is not defined

In [32]:
dic = {}
for i in range(2):
    dic[i] = {}
    dic[i][i + 1] = -1
    
print(dic[1][2])

-1


In [23]:
edgeL = list(g.edges)
print(len(edgeL))
for u, v, i in edgeL:
    if u == 7276 and v == 35:
        print(u, v)
    # print(g[u][v][0]['time'])
    # break
# print(g)
# g[edgeL[0][0]][edgeL[0][1]][0]

9974


In [ ]:
# Dijkstra Old
def Dijkstra(k: networkx.MultiDiGraph, startingIndex = 0):
    unvisited = list(k.nodes)
    visited = [unvisited.pop(startingIndex)]

    if len(list(networkx.neighbors(k, visited[-1]))) == 0:
        # print("No path to any other node found")
        return
        
    shortestPath = {}
    for node in list(k.nodes):
        if node == visited[0]:
            shortestPath[node] = (node, 0)
        else:
            shortestPath[node] = (-1, -1)
            
    # print(shortestPath)

    while len(unvisited) > 0:
        nodeToConsider = visited[-1]
        
                
        # Update shortestPath
        for node in unvisited:
            dis = shortestPath[node][1]
            
            # Skip node that cant be reached directly
            if networkx.is_path(k, [nodeToConsider, node]) == False:
                continue
            
            weighToHere = shortestPath[nodeToConsider][1]
            minWeight = k[nodeToConsider][node][0]['time']
            if dis == -1:
                shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
            elif shortestPath[node][1] > minWeight + weighToHere:
                    shortestPath[node] = (nodeToConsider, minWeight + weighToHere)
        
        nextNodeToConsider = -1
        minWeightSoFar = -1
        for u in unvisited:
            # Skip node that are not updated yet
            if shortestPath[u][1] == -1:
                continue
            
            # print(shortestPath[u][1])
            if minWeightSoFar == -1:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
            elif shortestPath[u][1] < minWeightSoFar:
                minWeightSoFar = shortestPath[u][1]
                nextNodeToConsider = u
                
        if nextNodeToConsider == -1:
            # print("Unreachable node found")
            return shortestPath
            
        
        # print(shortestPath)
        # print(f"{nextNodeToConsider=}")
        
        unvisited.remove(nextNodeToConsider)
        visited.append(nextNodeToConsider)
        # break

    # print()
    return shortestPath

Dijkstra(g)

In [ ]:
from rich import print
# print(g.subgraph(list(connected_components(ug))[0]))
grph: networkx.MultiDiGraph = g.subgraph(list(connected_components(ug))[0]).copy()
stops = list(grph.nodes)
stops.sort()
print(len(stops))